# Transfer Learning in Tensorflow with Inception V3

## Introduction

Transfer learning is the process of taking a pre-trained model (the weights and parameters of a network that has been trained on a large dataset by somebody else) and “fine-tuning” the model with your own dataset. The idea is that this pre-trained model will act as a feature extractor. You will remove the last layer of the network and replace it with your own classifier (depending on what your problem space is). You then freeze the weights of all the other layers and train the network normally (Freezing the layers means not changing the weights during gradient descent/optimization).

For this experiment we used Google's Inception-V3 pretrained model for Image Classification. This model consists of two parts:
    - Feature extraction part with a convolutional neural network.
    - Classification part with fully-connected and softmax layers.
The pre-trained Inception-v3 model achieves state-of-the-art accuracy for recognizing general objects with 1000 classes. The model extracts general features from input images in the first part and classifies them based on those features in the second part.

We will use this pre-trained model and re-train it it to classify aerial views of houses which have a swimming pool in their backyard. In order to do so we will build a new image dataset with aerial views coming from Goolgle Images. 

The following chart shows how the data flows in the Inception v3 model, which is a Convolutional Neural Network with many layers and a complicated structure. 

<img src="images/inception_flowchart.png">

In transfer learning, when you build a new model to classify your original dataset, you reuse the feature extraction part and re-train the classification part with your dataset. Since you don't have to train the feature extraction part (which is the most complex part of the model), you can train the model with less computational resources and training time.

<img src="images/inception_training.png">

The next step is to train the classification part of the model using the preprocessed data. The previous diagram shows the relationship between the preprocessing and the training.

## Disclaimer

This demo could not have been built without the research and work done by **Aniruddha Tapas**<br> and published on Github:https://github.com/Aniruddha-Tapas

The content of the following paper: http://ijarcet.org/wp-content/uploads/IJARCET-VOL-5-ISSUE-11-2664-2669.pdf has largely driven the execution of the demo as well as the fork of his github repository at: https://github.com/Aniruddha-Tapas/Transfer-Learning-for-Animal-Classification-in-Tensorflow

The images used in this demo application have been downloaded from publicly available images on the internet, and the copyright belongs to the respective owners. 


## Dataset

For this experiment, we built two small image dataset (~350 images) from the Google Images Web site. One with aerial view of houses without swimming pool and another one with aerial view of houses with a swimming pool in their backyard. 
To extract and load the pictures, we used 'googliser.sh' (https://github.com/teracow/googliser.git). 'googliser.sh'is a BASH script to perform fast image downloads sourced from Google Images based upon a user-specified search-phrase. It's a web-page scraper that feeds a list of image URLs to Wget to download images in parallel then combines them into a gallery image. 

Commands to build the two datasets: 

**./googliser.sh -p "house pool aerial view" -f 0 -g -n 500**<br>
**./googliser.sh -p "house aerial view" -f 0 -g -n 500**

After downloading the images, we took an extra step to visualize the images and remove the false positive. All the images are then saved in two different directories.

Next step is to resize all to images to 299x299.

Install: **pip install python-resize-image** 



## Data Preprocessing

The images saved in the directory may not be optimal for classifying using transfer learning on the Inception model as it requires 299 X 299 pixel sizes. We also rename the images for easier manipulation. Copy and run the following code in the image directory.

### Renaming and Resizing Images

In [6]:
#! python2.7
import os
from PIL import Image
from resizeimage import resizeimage

# Files downloaded from the Goolgle Images Web site follow a naming convention that includes '(xxxx)'   
# to number the images (i.e. google-image(0240).jpeg). The two lines below remove the parentheses.
[os.rename(f, f.replace('(', '-')) for f in os.listdir('.') if f.endswith('.jpeg')]
[os.rename(f, f.replace(')', '')) for f in os.listdir('.') if f.endswith('.jpeg')]

# Resize the images to 299x299  
def resize_file(in_file):
    fd_img = open(in_file, 'r')
    img = Image.open(fd_img)
    img = resizeimage.resize_contain(img, [299, 299])
    img.save((in_file.rsplit( ".", 1 )[ 0 ]) + '-resized.jpeg', img.format)
    fd_img.close()
    os.remove(in_file)

[resize_file(f)  for f in os.listdir('.') if f.endswith('.jpeg')]


[]

This **Python 2.7 **script resizes the images to 299 x 299 by padding white color to the boundaries.

For example it resizes 

<img src="images/google-image-0002.jpeg">

to : 

<img src="images/google-image-0002-resized.jpeg">

Now that we have our datasets ready we can move over to actually code our image classifier.

# Visualize dataset images

## Houses
Range of images are between **0001** and **0511**. As some 'false positive' have been removed as well as 'garbage' images, not all the range is covered. For example, 0024, 0217, 0432 exist but not 0047 or 0410 or ...so try different image numbers.<br><br>
To visualize a different image, double click on the displayed image below, the command will show up. Change the image number to display another one.


<img src="House-Pool/House/google-image-0001-resized.jpeg">

## Houses + Pools
Range of images are between **0001** and **0504**. As some 'false positive' have been removed as well as 'garbage' images, not all the range is covered. For example, 0113, 0285, 0467 exist but not 0105 or 0302 or ...so try different image numbers.<br><br>
To visualize a different image, double click on the displayed image below, the command will show up. Change the image number to display another one.


<img src="House-Pool/Pool/google-image-0471-resized.jpeg">

# Retraining

The following scripts demonstrate how to take an Inception v3 architecture model trained on
ImageNet images, and train a new top layer that can recognize other classes of
images.

The top layer receives as input a 2048-dimensional vector for each image. We
train a softmax layer on top of this representation. Assuming the softmax layer
contains N labels, this corresponds to learning N + 2048*N model parameters
corresponding to the learned biases and weights.

We have a folder called **House-Pool** and two subfolders called **Pool** and **House** containing each one a different set of images (with and without pools).<br> 
The subfolder names are important, since they define what label is applied to each image, but the filenames themselves don't matter. The label for each image is taken from the name of the subfolder it's in. This produces a new model file that can be loaded and run by any TensorFlow program, for example the label_image sample code.

## Imports

In [7]:
import sys
sys.path.insert(0, "/opt/DL/tensorflow/lib/python2.7/site-packages/")

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import hashlib
import os.path
import random
import re
import struct
import sys
import tarfile

import numpy as np
from six.moves import urllib
import tensorflow as tf

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.platform import gfile
from tensorflow.python.util import compat

from shutil import copyfile

## Parameters

The following are all parameters that are tied to the particular model architecture
we're using for Inception v3. These include things like tensor names and their
sizes. If you want to adapt this script to work with another model, you will
need to update these to reflect the values in the network you're using.

In [8]:
import os
image_dir = os.path.join("House-Pool")
output_graph = "output_graph.pb"
output_labels = "output_labels.txt"
output_graph_orig = "output_graph_orig.pb"
summaries_dir = "/tmp/output_labels.txt"
# how_many_training_steps = 500
how_many_training_steps = 500
learning_rate = 0.01
testing_percentage = 10
validation_percentage = 10
eval_step_interval = 10
train_batch_size = 100
test_batch_size = -1
validation_batch_size = 100
print_misclassified_test_images = False
model_dir = os.path.join('inception')
bottleneck_dir = "bottlenecks"
final_tensor_name = "final_result"
flip_left_right = False
random_crop = 0
random_scale = 0
random_brightness = 0
DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'

# pylint: enable=line-too-long
BOTTLENECK_TENSOR_NAME = 'pool_3/_reshape:0'
BOTTLENECK_TENSOR_SIZE = 2048
MODEL_INPUT_WIDTH = 299
MODEL_INPUT_HEIGHT = 299
MODEL_INPUT_DEPTH = 3
JPEG_DATA_TENSOR_NAME = 'DecodeJpeg/contents:0'
RESIZED_INPUT_TENSOR_NAME = 'ResizeBilinear:0'
MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 - 1  # ~134M

## Definitions

In [9]:
def create_image_lists(image_dir, testing_percentage, validation_percentage):
  if not gfile.Exists(image_dir):
    print("Image directory '" + image_dir + "' not found.")
    return None
  result = {}
  sub_dirs = [x[0] for x in gfile.Walk(image_dir)]
  # The root directory comes first, so skip it.
  is_root_dir = True
  for sub_dir in sub_dirs:
    if is_root_dir:
      is_root_dir = False
      continue
    extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']
    file_list = []
    dir_name = os.path.basename(sub_dir)
    if dir_name == image_dir:
      continue
    print("Looking for images in '" + dir_name + "'")
    for extension in extensions:
      file_glob = os.path.join(image_dir, dir_name, '*.' + extension)
      file_list.extend(gfile.Glob(file_glob))
    if not file_list:
      print('No files found')
      continue
    if len(file_list) < 20:
      print('WARNING: Folder has less than 20 images, which may cause issues.')
    elif len(file_list) > MAX_NUM_IMAGES_PER_CLASS:
      print('WARNING: Folder {} has more than {} images. Some images will '
            'never be selected.'.format(dir_name, MAX_NUM_IMAGES_PER_CLASS))
    label_name = re.sub(r'[^a-z0-9]+', ' ', dir_name.lower())
    training_images = []
    testing_images = []
    validation_images = []
    for file_name in file_list:
      base_name = os.path.basename(file_name)
      # We want to ignore anything after '_nohash_' in the file name when
      # deciding which set to put an image in, the data set creator has a way of
      # grouping photos that are close variations of each other. For example
      # this is used in the plant disease data set to group multiple pictures of
      # the same leaf.
      hash_name = re.sub(r'_nohash_.*$', '', file_name)
      # This looks a bit magical, but we need to decide whether this file should
      # go into the training, testing, or validation sets, and we want to keep
      # existing files in the same set even if more files are subsequently
      # added.
      # To do that, we need a stable way of deciding based on just the file name
      # itself, so we do a hash of that and then use that to generate a
      # probability value that we use to assign it.
      hash_name_hashed = hashlib.sha1(compat.as_bytes(hash_name)).hexdigest()
      percentage_hash = ((int(hash_name_hashed, 16) %
                          (MAX_NUM_IMAGES_PER_CLASS + 1)) *
                         (100.0 / MAX_NUM_IMAGES_PER_CLASS))
      if percentage_hash < validation_percentage:
        validation_images.append(base_name)
      elif percentage_hash < (testing_percentage + validation_percentage):
        testing_images.append(base_name)
      else:
        training_images.append(base_name)
    result[label_name] = {
        'dir': dir_name,
        'training': training_images,
        'testing': testing_images,
        'validation': validation_images,
    }
  return result



def get_image_path(image_lists, label_name, index, image_dir, category):
  if label_name not in image_lists:
    tf.logging.fatal('Label does not exist %s.', label_name)
  label_lists = image_lists[label_name]
  if category not in label_lists:
    tf.logging.fatal('Category does not exist %s.', category)
  category_list = label_lists[category]
  if not category_list:
    tf.logging.fatal('Label %s has no images in the category %s.',
                     label_name, category)
  mod_index = index % len(category_list)
  base_name = category_list[mod_index]
  sub_dir = label_lists['dir']
  full_path = os.path.join(image_dir, sub_dir, base_name)
  return full_path


def get_bottleneck_path(image_lists, label_name, index, bottleneck_dir,
                        category):
  
  return get_image_path(image_lists, label_name, index, bottleneck_dir,
                        category) + '.txt'


def create_inception_graph():
  with tf.Session() as sess:
    model_filename = os.path.join(
        model_dir, 'classify_image_graph_def.pb')
    with gfile.FastGFile(model_filename, 'rb') as f:
      graph_def = tf.GraphDef()
      graph_def.ParseFromString(f.read())
      bottleneck_tensor, jpeg_data_tensor, resized_input_tensor = (
          tf.import_graph_def(graph_def, name='', return_elements=[
              BOTTLENECK_TENSOR_NAME, JPEG_DATA_TENSOR_NAME,
              RESIZED_INPUT_TENSOR_NAME]))
  return sess.graph, bottleneck_tensor, jpeg_data_tensor, resized_input_tensor


def run_bottleneck_on_image(sess, image_data, image_data_tensor,
                            bottleneck_tensor):
  bottleneck_values = sess.run(
      bottleneck_tensor,
      {image_data_tensor: image_data})
  bottleneck_values = np.squeeze(bottleneck_values)
  return bottleneck_values


def maybe_download_and_extract():
  dest_directory = model_dir
  if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
  filename = DATA_URL.split('/')[-1]
  filepath = os.path.join(dest_directory, filename)
  if not os.path.exists(filepath):

    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' %
                       (filename,
                        float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()

    filepath, _ = urllib.request.urlretrieve(DATA_URL,
                                             filepath,
                                             _progress)
    print()
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
  tarfile.open(filepath, 'r:gz').extractall(dest_directory)


def ensure_dir_exists(dir_name):
  if not os.path.exists(dir_name):
    os.makedirs(dir_name)


def write_list_of_floats_to_file(list_of_floats , file_path):
  s = struct.pack('d' * BOTTLENECK_TENSOR_SIZE, *list_of_floats)
  with open(file_path, 'wb') as f:
    f.write(s)


def read_list_of_floats_from_file(file_path):

  with open(file_path, 'rb') as f:
    s = struct.unpack('d' * BOTTLENECK_TENSOR_SIZE, f.read())
    return list(s)


bottleneck_path_2_bottleneck_values = {}

def create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor, bottleneck_tensor):
  print('Creating bottleneck at ' + bottleneck_path)
  image_path = get_image_path(image_lists, label_name, index, image_dir, category)
  if not gfile.Exists(image_path):
    tf.logging.fatal('File does not exist %s', image_path)
  image_data = gfile.FastGFile(image_path, 'rb').read()
  bottleneck_values = run_bottleneck_on_image(sess, image_data, jpeg_data_tensor, bottleneck_tensor)
  bottleneck_string = ','.join(str(x) for x in bottleneck_values)
  with open(bottleneck_path, 'w') as bottleneck_file:
    bottleneck_file.write(bottleneck_string)


def get_or_create_bottleneck(sess, image_lists, label_name, index, image_dir,
                             category, bottleneck_dir, jpeg_data_tensor,
                             bottleneck_tensor):
  label_lists = image_lists[label_name]
  sub_dir = label_lists['dir']
  sub_dir_path = os.path.join(bottleneck_dir, sub_dir)
  ensure_dir_exists(sub_dir_path)
  bottleneck_path = get_bottleneck_path(image_lists, label_name, index, bottleneck_dir, category)
  if not os.path.exists(bottleneck_path):
    create_bottleneck_file(bottleneck_path, image_lists, label_name, index, image_dir, category, sess, jpeg_data_tensor, bottleneck_tensor)
  with open(bottleneck_path, 'r') as bottleneck_file:
    bottleneck_string = bottleneck_file.read()
  did_hit_error = False
  try:
    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
  except:
    print("Invalid float found, recreating bottleneck")
    did_hit_error = True
  if did_hit_error:
    create_bottleneck_file(bottleneck_path, image_lists, label_name, index, image_dir, category, sess, jpeg_data_tensor, bottleneck_tensor)
    with open(bottleneck_path, 'r') as bottleneck_file:
      bottleneck_string = bottleneck_file.read()
    # Allow exceptions to propagate here, since they shouldn't happen after a fresh creation
    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
  return bottleneck_values


def cache_bottlenecks(sess, image_lists, image_dir, bottleneck_dir,
                      jpeg_data_tensor, bottleneck_tensor):
  how_many_bottlenecks = 0
  ensure_dir_exists(bottleneck_dir)
  for label_name, label_lists in image_lists.items():
    for category in ['training', 'testing', 'validation']:
      category_list = label_lists[category]
      for index, unused_base_name in enumerate(category_list):
        get_or_create_bottleneck(sess, image_lists, label_name, index,
                                 image_dir, category, bottleneck_dir,
                                 jpeg_data_tensor, bottleneck_tensor)

        how_many_bottlenecks += 1
        if how_many_bottlenecks % 100 == 0:
          print(str(how_many_bottlenecks) + ' bottleneck files created.')


def get_random_cached_bottlenecks(sess, image_lists, how_many, category,
                                  bottleneck_dir, image_dir, jpeg_data_tensor,
                                  bottleneck_tensor):
  class_count = len(image_lists.keys())
  bottlenecks = []
  ground_truths = []
  filenames = []
  if how_many >= 0:
    # Retrieve a random sample of bottlenecks.
    for unused_i in range(how_many):
      label_index = random.randrange(class_count)
      label_name = list(image_lists.keys())[label_index]
      image_index = random.randrange(MAX_NUM_IMAGES_PER_CLASS + 1)
      image_name = get_image_path(image_lists, label_name, image_index,
                                  image_dir, category)
      bottleneck = get_or_create_bottleneck(sess, image_lists, label_name,
                                            image_index, image_dir, category,
                                            bottleneck_dir, jpeg_data_tensor,
                                            bottleneck_tensor)
      ground_truth = np.zeros(class_count, dtype=np.float32)
      ground_truth[label_index] = 1.0
      bottlenecks.append(bottleneck)
      ground_truths.append(ground_truth)
      filenames.append(image_name)
  else:
    # Retrieve all bottlenecks.
    for label_index, label_name in enumerate(image_lists.keys()):
      for image_index, image_name in enumerate(
          image_lists[label_name][category]):
        image_name = get_image_path(image_lists, label_name, image_index,
                                    image_dir, category)
        bottleneck = get_or_create_bottleneck(sess, image_lists, label_name,
                                              image_index, image_dir, category,
                                              bottleneck_dir, jpeg_data_tensor,
                                              bottleneck_tensor)
        ground_truth = np.zeros(class_count, dtype=np.float32)
        ground_truth[label_index] = 1.0
        bottlenecks.append(bottleneck)
        ground_truths.append(ground_truth)
        filenames.append(image_name)
  return bottlenecks, ground_truths, filenames


def get_random_distorted_bottlenecks(
    sess, image_lists, how_many, category, image_dir, input_jpeg_tensor,
    distorted_image, resized_input_tensor, bottleneck_tensor):

  class_count = len(image_lists.keys())
  bottlenecks = []
  ground_truths = []
  for unused_i in range(how_many):
    label_index = random.randrange(class_count)
    label_name = list(image_lists.keys())[label_index]
    image_index = random.randrange(MAX_NUM_IMAGES_PER_CLASS + 1)
    image_path = get_image_path(image_lists, label_name, image_index, image_dir,
                                category)
    if not gfile.Exists(image_path):
      tf.logging.fatal('File does not exist %s', image_path)
    jpeg_data = gfile.FastGFile(image_path, 'rb').read()
    # Note that we materialize the distorted_image_data as a numpy array before
    # sending running inference on the image. This involves 2 memory copies and
    # might be optimized in other implementations.
    distorted_image_data = sess.run(distorted_image,
                                    {input_jpeg_tensor: jpeg_data})
    bottleneck = run_bottleneck_on_image(sess, distorted_image_data,
                                         resized_input_tensor,
                                         bottleneck_tensor)
    ground_truth = np.zeros(class_count, dtype=np.float32)
    ground_truth[label_index] = 1.0
    bottlenecks.append(bottleneck)
    ground_truths.append(ground_truth)
  return bottlenecks, ground_truths



def should_distort_images(flip_left_right, random_crop, random_scale,
                          random_brightness):
  return (flip_left_right or (random_crop != 0) or (random_scale != 0) or
          (random_brightness != 0))



def add_input_distortions(flip_left_right, random_crop, random_scale,
                          random_brightness):
  
  jpeg_data = tf.placeholder(tf.string, name='DistortJPGInput')
  decoded_image = tf.image.decode_jpeg(jpeg_data, channels=MODEL_INPUT_DEPTH)
  decoded_image_as_float = tf.cast(decoded_image, dtype=tf.float32)
  decoded_image_4d = tf.expand_dims(decoded_image_as_float, 0)
  margin_scale = 1.0 + (random_crop / 100.0)
  resize_scale = 1.0 + (random_scale / 100.0)
  margin_scale_value = tf.constant(margin_scale)
  resize_scale_value = tf.random_uniform(tensor_shape.scalar(),
                                         minval=1.0,
                                         maxval=resize_scale)
  scale_value = tf.multiply(margin_scale_value, resize_scale_value)
  precrop_width = tf.multiply(scale_value, MODEL_INPUT_WIDTH)
  precrop_height = tf.multiply(scale_value, MODEL_INPUT_HEIGHT)
  precrop_shape = tf.stack([precrop_height, precrop_width])
  precrop_shape_as_int = tf.cast(precrop_shape, dtype=tf.int32)
  precropped_image = tf.image.resize_bilinear(decoded_image_4d,
                                              precrop_shape_as_int)
  precropped_image_3d = tf.squeeze(precropped_image, squeeze_dims=[0])
  cropped_image = tf.random_crop(precropped_image_3d,
                                 [MODEL_INPUT_HEIGHT, MODEL_INPUT_WIDTH,
                                  MODEL_INPUT_DEPTH])
  if flip_left_right:
    flipped_image = tf.image.random_flip_left_right(cropped_image)
  else:
    flipped_image = cropped_image
  brightness_min = 1.0 - (random_brightness / 100.0)
  brightness_max = 1.0 + (random_brightness / 100.0)
  brightness_value = tf.random_uniform(tensor_shape.scalar(),
                                       minval=brightness_min,
                                       maxval=brightness_max)
  brightened_image = tf.multiply(flipped_image, brightness_value)
  distort_result = tf.expand_dims(brightened_image, 0, name='DistortResult')
  return jpeg_data, distort_result



def variable_summaries(var):
  
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)



def add_final_training_ops(class_count, final_tensor_name, bottleneck_tensor):
  
  with tf.name_scope('input'):
    bottleneck_input = tf.placeholder_with_default(
        bottleneck_tensor, shape=[None, BOTTLENECK_TENSOR_SIZE],
        name='BottleneckInputPlaceholder')

    ground_truth_input = tf.placeholder(tf.float32,
                                        [None, class_count],
                                        name='GroundTruthInput')

  # Organizing the following ops as `final_training_ops` so they're easier
  # to see in TensorBoard
  layer_name = 'final_training_ops'
  with tf.name_scope(layer_name):
    with tf.name_scope('weights'):
      layer_weights = tf.Variable(tf.truncated_normal([BOTTLENECK_TENSOR_SIZE, class_count], stddev=0.001), name='final_weights')
      variable_summaries(layer_weights)
    with tf.name_scope('biases'):
      layer_biases = tf.Variable(tf.zeros([class_count]), name='final_biases')
      variable_summaries(layer_biases)
    with tf.name_scope('Wx_plus_b'):
      logits = tf.matmul(bottleneck_input, layer_weights) + layer_biases
      tf.summary.histogram('pre_activations', logits)

  final_tensor = tf.nn.softmax(logits, name=final_tensor_name)
  tf.summary.histogram('activations', final_tensor)

  with tf.name_scope('cross_entropy'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
        labels=ground_truth_input, logits=logits)
    with tf.name_scope('total'):
      cross_entropy_mean = tf.reduce_mean(cross_entropy)
  tf.summary.scalar('cross_entropy', cross_entropy_mean)

  with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(
        cross_entropy_mean)

  return (train_step, cross_entropy_mean, bottleneck_input, ground_truth_input,
          final_tensor)



def add_evaluation_step(result_tensor, ground_truth_tensor):

  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      prediction = tf.argmax(result_tensor, 1)
      correct_prediction = tf.equal(
          prediction, tf.argmax(ground_truth_tensor, 1))
    with tf.name_scope('accuracy'):
      evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', evaluation_step)
  return evaluation_step, prediction



## Main function


In [13]:
import shutil, sys

# do some cleanup
if os.path.isfile(output_graph):
    os.remove(output_graph)

if os.path.isdir(model_dir):    
    shutil.rmtree(model_dir, ignore_errors=False, onerror=None)
    
if os.path.isdir(bottleneck_dir):    
    shutil.rmtree(bottleneck_dir, ignore_errors=False, onerror=None)

tf.reset_default_graph()

# Setup the directory we'll write summaries to for TensorBoard
if tf.gfile.Exists(summaries_dir):
    tf.gfile.DeleteRecursively(summaries_dir)
tf.gfile.MakeDirs(summaries_dir)

# Set up the pre-trained graph.
maybe_download_and_extract()
graph, bottleneck_tensor, jpeg_data_tensor, resized_image_tensor = (
      create_inception_graph())

# Look at the folder structure, and create lists of all the images.
image_lists = create_image_lists(image_dir, testing_percentage,
                                   validation_percentage)
class_count = len(image_lists.keys())
if class_count == 0:
    print('No valid folders of images found at ' + image_dir)
if class_count == 1:
    print('Only one valid folder of images found at ' + image_dir +
          ' - multiple classes are needed for classification.')
# See if the command-line flags mean we're applying any distortions.
do_distort_images = should_distort_images(
      flip_left_right, random_crop, random_scale,
      random_brightness)
sess = tf.Session()

if do_distort_images:
    # We will be applying distortions, so setup the operations we'll need.
    distorted_jpeg_data_tensor, distorted_image_tensor = add_input_distortions(
        flip_left_right, random_crop, random_scale,
        random_brightness)
else:
    # We'll make sure we've calculated the 'bottleneck' image summaries and
    # cached them on disk.
    cache_bottlenecks(sess, image_lists, image_dir, bottleneck_dir,
                      jpeg_data_tensor, bottleneck_tensor)

# Add the new layer that we'll be training.
(train_step, cross_entropy, bottleneck_input, ground_truth_input,
   final_tensor) = add_final_training_ops(len(image_lists.keys()),
                                          final_tensor_name,
                                          bottleneck_tensor)

# Create the operations we need to evaluate the accuracy of our new layer.
evaluation_step, prediction = add_evaluation_step(
      final_tensor, ground_truth_input)

# Merge all the summaries and write them out to /tmp/retrain_logs (by default)
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(summaries_dir + '/train',
                                       sess.graph)
validation_writer = tf.summary.FileWriter(summaries_dir + '/validation')

# Set up all our weights to their initial default values.
init = tf.global_variables_initializer()
sess.run(init)

# FRB
output_graph_def = graph_util.convert_variables_to_constants(
    sess, graph.as_graph_def(), [final_tensor_name])
with gfile.FastGFile(output_graph_orig, 'wb') as f:
    f.write(output_graph_def.SerializeToString())

# Run the training for as many cycles as requested on the command line.
for i in range(how_many_training_steps):
    # Get a batch of input bottleneck values, either calculated fresh every time
    # with distortions applied, or from the cache stored on disk.
    if do_distort_images:
        train_bottlenecks, train_ground_truth = get_random_distorted_bottlenecks(
            sess, image_lists, train_batch_size, 'training',
            image_dir, distorted_jpeg_data_tensor,
            distorted_image_tensor, resized_image_tensor, bottleneck_tensor)
    else:
        train_bottlenecks, train_ground_truth, _ = get_random_cached_bottlenecks(
            sess, image_lists, train_batch_size, 'training',
            bottleneck_dir, image_dir, jpeg_data_tensor,
            bottleneck_tensor)
    # Feed the bottlenecks and ground truth into the graph, and run a training
    # step. Capture training summaries for TensorBoard with the `merged` op.
    train_summary, _ = sess.run([merged, train_step],
                                feed_dict={bottleneck_input: train_bottlenecks,
                                           ground_truth_input: train_ground_truth})
    train_writer.add_summary(train_summary, i)

    # Every so often, print out how well the graph is training.
    is_last_step = (i + 1 == how_many_training_steps)
    if (i % eval_step_interval) == 0 or is_last_step:
        train_accuracy, cross_entropy_value = sess.run(
            [evaluation_step, cross_entropy],
            feed_dict={bottleneck_input: train_bottlenecks,
                       ground_truth_input: train_ground_truth})
        print('%s: Step %d: Train accuracy = %.1f%%' % (datetime.now(), i,
                                                      train_accuracy * 100))
        print('%s: Step %d: Cross entropy = %f' % (datetime.now(), i,
                                                 cross_entropy_value))
        validation_bottlenecks, validation_ground_truth, _ = (
            get_random_cached_bottlenecks(
                sess, image_lists, validation_batch_size, 'validation',
                bottleneck_dir, image_dir, jpeg_data_tensor,
                bottleneck_tensor))
        # Run a validation step and capture training summaries for TensorBoard
        # with the `merged` op.
        validation_summary, validation_accuracy = sess.run(
            [merged, evaluation_step],
            feed_dict={bottleneck_input: validation_bottlenecks,
                     ground_truth_input: validation_ground_truth})
        validation_writer.add_summary(validation_summary, i)
        print('%s: Step %d: Validation accuracy = %.1f%% (N=%d)' %
              (datetime.now(), i, validation_accuracy * 100,
               len(validation_bottlenecks)))

# We've completed all our training, so run a final test evaluation on
# some new images we haven't used before.
test_bottlenecks, test_ground_truth, test_filenames = (
    get_random_cached_bottlenecks(sess, image_lists, test_batch_size,
                                  'testing', bottleneck_dir,
                                  image_dir, jpeg_data_tensor,
                                  bottleneck_tensor))
test_accuracy, predictions = sess.run(
    [evaluation_step, prediction],
    feed_dict={bottleneck_input: test_bottlenecks,
               ground_truth_input: test_ground_truth})
print('Final test accuracy = %.1f%% (N=%d)' % (
    test_accuracy * 100, len(test_bottlenecks)))

if print_misclassified_test_images:
    print('=== MISCLASSIFIED TEST IMAGES ===')
    for i, test_filename in enumerate(test_filenames):
        if predictions[i] != test_ground_truth[i].argmax():
            print('%70s  %s' % (test_filename, image_lists.keys()[predictions[i]]))

# Write out the trained graph and labels with the weights stored as constants.
output_graph_def = graph_util.convert_variables_to_constants(
    sess, graph.as_graph_def(), [final_tensor_name])
with gfile.FastGFile(output_graph, 'wb') as f:
    f.write(output_graph_def.SerializeToString())
with gfile.FastGFile(output_labels, 'w') as f:
    f.write('\n'.join(image_lists.keys()) + '\n')
    


>> Downloading inception-2015-12-05.tgz 100.0%
Successfully downloaded inception-2015-12-05.tgz 88931400 bytes.
Looking for images in 'House'
Looking for images in 'Pool'
Creating bottleneck at bottlenecks/House/google-image-0482-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0014-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0104-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0225-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0432-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0005-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0259-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0107-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0051-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0433-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0354-resized.jp

Creating bottleneck at bottlenecks/House/google-image-0019-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0024-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0262-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0434-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0323-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0511-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0142-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0155-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0290-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0420-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0079-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0330-resized.jpeg.txt
Creating bottleneck at bottlenecks/House/google-image-0021-resized.jpeg.txt
Creating bot

Creating bottleneck at bottlenecks/Pool/google-image-0015-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0178-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0437-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0388-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0471-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0343-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0075-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0254-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0174-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0176-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0228-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0194-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0348-resized.jpeg.txt
Creating bottleneck at bo

Creating bottleneck at bottlenecks/Pool/google-image-0181-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0123-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0265-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0424-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0087-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0327-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0054-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0445-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0240-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0200-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0271-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0273-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0172-resized.jpeg.txt
Creating bottleneck at bo

Creating bottleneck at bottlenecks/Pool/google-image-0322-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0017-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0090-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0263-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0394-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0029-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0488-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0270-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0030-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0250-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0366-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0244-resized.jpeg.txt
Creating bottleneck at bottlenecks/Pool/google-image-0213-resized.jpeg.txt
Creating bottleneck at bo

2017-06-07 17:01:19.905741: Step 120: Train accuracy = 90.0%
2017-06-07 17:01:19.906031: Step 120: Cross entropy = 0.335273
2017-06-07 17:01:19.991421: Step 120: Validation accuracy = 84.0% (N=100)
2017-06-07 17:01:20.877248: Step 130: Train accuracy = 86.0%
2017-06-07 17:01:20.877456: Step 130: Cross entropy = 0.353311
2017-06-07 17:01:20.963697: Step 130: Validation accuracy = 87.0% (N=100)
2017-06-07 17:01:21.860274: Step 140: Train accuracy = 85.0%
2017-06-07 17:01:21.860467: Step 140: Cross entropy = 0.352380
2017-06-07 17:01:21.947969: Step 140: Validation accuracy = 82.0% (N=100)
2017-06-07 17:01:22.832465: Step 150: Train accuracy = 94.0%
2017-06-07 17:01:22.832592: Step 150: Cross entropy = 0.290812
2017-06-07 17:01:22.918597: Step 150: Validation accuracy = 90.0% (N=100)
2017-06-07 17:01:23.809970: Step 160: Train accuracy = 93.0%
2017-06-07 17:01:23.810112: Step 160: Cross entropy = 0.291722
2017-06-07 17:01:23.896830: Step 160: Validation accuracy = 91.0% (N=100)
2017-06-07

The final test accuracy is **~85%**  for our two classes **House** and **House + Pool** which is quite substantial given our training set contained approxiamtely only ~300 images for each classes. This is where Transfer Learning really shines. All what we did was to use the trained Inception Model which already had learned basic features of lines, shapes and other features that increase in abstraction as we move towards the final layers of the model. We basically retrained the last layers where we supplied training images of Houses and House + Pools and the model using its pre-learnt features.  

## Want to give it a try ?

We have a couple of images you can use to test the model or you can download your owns from your favorite web site.

**image-01-resized.jpeg**  <img src="test_images/image-01.jpeg" >
**image-02-resized.jpeg**  <img src="test_images/image-02.jpeg" >
**image-03-resized.jpeg**  <img src="test_images/image-03.jpeg" >
**image-04-resized.jpeg**  <img src="test_images/image-04.jpeg" >
**image-05-resized.jpeg**  <img src="test_images/image-05.jpeg" >
**image-06-resized.jpeg**  <img src="test_images/image-06.jpeg" >
**image-07-resized.jpeg**  <img src="test_images/image-07.jpeg" >


### Run the inference engine
Open a terminal window form the Jupyter Notebook. <br>
The **samples** directory includes some test images. 

<img src="images/terminal-access.png" >

To run the inference engine use the following command: <br><br>
**python test-new.py test_images/image-01.jpeg**
<br><br>
You can also download your own set of images and run them against the inference engine.
<br>

### Run the inference engine against the **original** Inception V3 model

In [20]:
%run test-orig.py test_images/image-03.jpeg

house (score = 0.51582)
house + pool (score = 0.48418)


This is the expected result as the Inception V3 model has not been trained with the new image dataset.

### Run the inference engine against new model

In [21]:
%run test-new.py test_images/image-03.jpeg

house + pool (score = 0.99756)
house (score = 0.00244)


Yeah, the new model is able to properly classify the images...

## Conclusion
I hope that you would now be able to apply pre-trained models to your problem statements. Be sure that the pre-trained model you have selected has been trained on a similar data set as the one that you wish to use it on. There are various architectures people have tried on different types of data sets and I strongly encourage you to go through these architectures and apply them on your own problem statements.
